In [1]:
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

In [2]:
import os
import random
import io
import numpy as np
import scipy as sp
import pandas as pd
from collections import Counter
from os import listdir
from os.path import isfile, join, isdir
from random import shuffle
import fitz
import re
import ast

In [3]:
documents_path=r'E:\planning_validator_data\raw_data\sorted_doc_types\image_based\new_download-15-2-22\for_labelling_azure'

In [4]:
label_files=listdir('labels')
def read_csv(file):
    return pd.read_csv('labels\\'+file)
labels=map(read_csv,label_files)
labels=pd.concat(list(labels))
labels=labels.dropna(subset=['Labels'])

In [18]:
documents = []
targets = []
min_instance_len_in_characters=1

for idx, row in labels.iterrows():
    for folder in listdir(documents_path):
        for file in listdir(join(documents_path,folder)):
            if isfile(join(documents_path,folder,file)):
                if row[0] == file:
                    if file.endswith('.pdf'):
                        filename_path = join(documents_path,folder,file)
                        doc=fitz.open(filename_path)
                        token_text=''
                        for page in doc:
                            text = page.get_text('text').strip()
                            token_text = token_text+text
                            text=''
                        if len(token_text)>min_instance_len_in_characters:
                            documents.append(token_text)
                            #print(folder,file)
                            try:
                                targets.append(ast.literal_eval(row[1]))
                            except:
                                print(folder,file)

In [21]:
%%time

import warnings
import scipy as sp
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize as normalize_func
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
#from xgboost import XGBClassifier
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from vectorizer import TfIdfVectorizer
from vectorizer import CompositeTextVectorizer

def training_pipeline(documents,targets,clf):
    mlb = MultiLabelBinarizer()
    mlb.fit(targets)
    y=mlb.transform(targets)

    # split train test
    documents_train, documents_valid, y_train, y_valid = train_test_split(documents, y, test_size=0.3, random_state=42)    
    vectorizer=TfidfVectorizer(stop_words='english', analyzer='char_wb', ngram_range=(3,7)).fit(documents_train)
    vectors=vectorizer.transform(documents_train)
    svd=TruncatedSVD(n_components=200, random_state=42).fit(vectors)
    truncated=svd.transform(vectors)
    x_train=normalize_func(truncated)

    model=clf.fit(x_train, y_train)

    vectors_valid=vectorizer.transform(documents_valid)
    trunc_valid=svd.transform(vectors_valid)
    x_valid=normalize_func(trunc_valid)
    preds=model.predict(x_valid)

    score=model.score(x_valid,y_valid)
    lrap_score=label_ranking_average_precision_score(y_valid, preds)

    return model,mlb,vectorizer,svd,score,lrap_score

CPU times: total: 0 ns
Wall time: 0 ns


In [22]:
clf_svc_weighted = MultiOutputClassifier(OneVsRestClassifier(SVC(C=100, kernel='rbf', gamma='scale', probability=True, class_weight='balanced')))
model_svc,mlb_svc,vectorizer_svc,svd_svc,score_svc,lrap_score_svc = training_pipeline(documents,targets,clf_svc_weighted)

C:\Users\rucha.dalwadi.AGILESAAS\Anaconda3\envs\objdet-venv\lib\site-packages\sklearn\feature_extraction\text.py:538: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
C:\Users\rucha.dalwadi.AGILESAAS\Anaconda3\envs\objdet-venv\lib\site-packages\sklearn\multiclass.py:79: UserWarning: Label not 0 is present in all training examples.
  warnings.warn(
C:\Users\rucha.dalwadi.AGILESAAS\Anaconda3\envs\objdet-venv\lib\site-packages\sklearn\multiclass.py:79: UserWarning: Label not 0 is present in all training examples.
  warnings.warn(
C:\Users\rucha.dalwadi.AGILESAAS\Anaconda3\envs\objdet-venv\lib\site-packages\sklearn\multiclass.py:79: UserWarning: Label not 0 is present in all training examples.
  warnings.warn(
C:\Users\rucha.dalwadi.AGILESAAS\Anaconda3\envs\objdet-venv\lib\site-packages\sklearn\multiclass.py:79: UserWarning: Label not 0 is present in all training examples.
  warnings.warn(
C:\Users\rucha.dalwadi.AGILESAAS\Anaconda3\envs\o

In [23]:
print(f'Score: {score_svc:.4f} | Label ranking average precision score: {lrap_score_svc:.4f}')

Score: 0.1707 | Label ranking average precision score: 0.3669
